In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 100% !important; }</style>"))

# Naive Bayes Classifier
We use the Naive Bayes Classifier to determine whether a job posting is fraudulent or real.

In [2]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
#import itertools

In [3]:
df = pd.read_excel('job_postings_excel.xlsx')

## The features and the target class of the dataset

In [4]:
features = df.columns[1:df.columns.shape[0]-1].to_numpy()
print('features:', '\n', features, '\n')

target_class = df.columns[df.columns.shape[0]-1]
print('target class:', '\n', target_class)

features: 
 ['title' 'location' 'department' 'salary_range' 'company_profile'
 'description' 'requirements' 'benefits' 'telecommuting'
 'has_company_logo' 'has_questions' 'employment_type'
 'required_experience' 'required_education' 'industry' 'function'] 

target class: 
 fraudulent


## Compute priors of the job postings

In [5]:
class_labels, class_labels_counts = np.unique(df['fraudulent'], return_counts=True)

prior_prob_real = class_labels_counts[0] / np.sum(class_labels_counts)
prior_prob_fraud = 1 - prior_prob_real

print('Class labels: ', class_labels)
print('Class labels counts: ', class_labels_counts, '\n')
print('Prior probability job posting is REAL: ', prior_prob_real)
print('Prior probability job posting is FRAUDULENT: ', prior_prob_fraud)

Class labels:  [0 1]
Class labels counts:  [17014   866] 

Prior probability job posting is REAL:  0.9515659955257271
Prior probability job posting is FRAUDULENT:  0.04843400447427293


## Split dataset into training, validation and test datasets.

In [6]:
training_data = df.loc[:10728]
validation_data = df.loc[10729:16450]
test_data = df.loc[16451:]

print('training data example item:','\n\n', training_data.loc[0])

training data example item: 

 job_id                                                                 1
title                                                   Marketing Intern
location                                                   US,NY,NewYork
department                                                     Marketing
salary_range                                                         NaN
company_profile        We're Food52, and we've created a groundbreaki...
description            Food52, a fast-growing, James Beard Award-winn...
requirements           Experience with content management systems a m...
benefits                                                             NaN
telecommuting                                                          0
has_company_logo                                                       1
has_questions                                                          0
employment_type                                                    Other
required_experience 

## Helper Functions:

### (a) Compute fraudulent counts of each feature

In [7]:
# This function computes the fraudulent and real counts of a specific feature
# and groups them by unique feature values
def compute_fraud_real_counts(class_labels, feature_full_data, feature_unique_data):
    
    # Get index of the classes
    real_index = np.where(class_labels == 0)[0]
    fraud_index = np.where(class_labels == 1)[0]
    
    # Get the real and fraudulent features
    feature_real = feature_full_data[real_index]
    feature_fraud = feature_full_data[fraud_index]
    
    # Get unique real and fraudulent features
    unique_feature_real, real_counts =  np.unique(feature_real ,return_counts=True)
    unique_feature_fraud, fraud_counts =  np.unique(feature_fraud ,return_counts=True)
    
    # Get the indices
    real_bool = np.isin(feature_unique_data,unique_feature_real)
    real_index = np.where(real_bool == True)[0]
    fraud_bool = np.isin(feature_unique_data,unique_feature_fraud)
    fraud_index = np.where(fraud_bool == True)[0]
        
    # Initialise counts
    unique_real_counts = np.zeros(feature_unique_data.shape)
    unique_fraud_counts = np.zeros(feature_unique_data.shape)
    
    # create unique counts
    unique_real_counts[real_index] = real_counts
    unique_fraud_counts[fraud_index] = fraud_counts
    
    # create the pandas object
    result_object = pd.DataFrame({'Real':unique_real_counts,
                                 'Fraudulent':unique_fraud_counts},
                                 index =feature_unique_data )
    return result_object

### This function computes probabilities of unique values of each feature with respect to each class label and applies laplacian smoothing if necessary.

In [8]:
def compute_probabilities2(feature_values_fraud_counts):
    
    #Temporary values
    fraud_arr = feature_values_fraud_counts['Fraudulent']
    real_arr = feature_values_fraud_counts['Real']

    
    # Init the probabilities
    prob_real = np.zeros(feature_values_fraud_counts.shape[0])
    prob_fraud = np.zeros(feature_values_fraud_counts.shape[0])
    
    # (1)Find index of zero values for  laplacian smoothing
    fraud_zero_index = np.where( fraud_arr == 0)[0]
    real_zero_index = np.where(real_arr == 0)[0]
    
    # apply laplacian by adding a fake datapoint count to both counts of each class label
    feature_value_freq_laplacian_real = np.add(fraud_arr[real_zero_index],real_arr[real_zero_index])+2
    feature_value_freq_laplacian_fraud = np.add(fraud_arr[fraud_zero_index],real_arr[fraud_zero_index])+2
    #Calculate probabilites
    prob_real[real_zero_index] = (real_arr[real_zero_index]+1)/feature_value_freq_laplacian_real
    prob_fraud[fraud_zero_index] = (fraud_arr[fraud_zero_index]+1)/feature_value_freq_laplacian_fraud
    
    # (2)Find index of non-zero values
    fraud_nonzero_index = np.where( fraud_arr != 0)[0]
    real_nonzero_index = np.where(real_arr != 0)[0]
    
    #Calculate probabilites
    feature_value_freq_real = np.add(fraud_arr[real_nonzero_index],real_arr[real_nonzero_index])
    feature_value_freq_fraud = np.add(fraud_arr[fraud_nonzero_index],real_arr[fraud_nonzero_index])
    
    prob_real[real_nonzero_index] = real_arr[real_nonzero_index]/feature_value_freq_real
    prob_fraud[fraud_nonzero_index] = fraud_arr[fraud_nonzero_index]/feature_value_freq_fraud

    
    # Create pandas object
    result_object = pd.DataFrame({'RealProb':prob_real,
                                 'FraudulentProb':prob_fraud},
                                 index =feature_values_fraud_counts.index)
    return result_object


## (1) Title Feature

compute fraud counts of each unique title with respect to the class labels.

In [9]:
training_data_titles = training_data['title'].str.lower()
unique_titles = np.unique(training_data_titles)

titles_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_titles, unique_titles)
print(titles_and_fraud_counts.head())

                                                    Real  Fraudulent
$950/week truck drivers needed                       1.0         0.0
$950/week. truck drivers needed                      1.0         0.0
(assistant) accountant                               1.0         0.0
(internship) communication / e-marketing assistant   1.0         0.0
(internship) pr / event manager assistant            1.0         0.0


compute probabilities of each unique title with respect to the class labes.

In [10]:
titles_and_fraud_prob = compute_probabilities2(titles_and_fraud_counts)
print(titles_and_fraud_prob.head())

                                                    RealProb  FraudulentProb
$950/week truck drivers needed                           1.0        0.333333
$950/week. truck drivers needed                          1.0        0.333333
(assistant) accountant                                   1.0        0.333333
(internship) communication / e-marketing assistant       1.0        0.333333
(internship) pr / event manager assistant                1.0        0.333333


## (2) Location feature

compute fraud counts of each unique location with respect to the class lables.

In [11]:

training_data_locations = training_data['location'].str.lower()

bool_nan = training_data_locations.isnull()
nan_index = np.where(bool_nan == True)[0]

training_data_locations[nan_index] ="location unavailable"

unique_locations_new = np.unique(training_data_locations)
print("unique locations :",unique_locations_new.shape)

unique locations : (2178,)


In [12]:
#compute fraud counts of each unique location with respect to the class labels.
locations_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_locations, unique_locations_new)
print(locations_and_fraud_counts.head())

                         Real  Fraudulent
ae                        3.0         0.0
ae,az,abudhabi            1.0         0.0
ae,du                     4.0         0.0
ae,du,dubai              16.0         0.0
ae,du,dubaiinternetcity   2.0         0.0


compute the probabilities of each unique location with respect to the class labels.

In [13]:
locations_and_fraud_prob = compute_probabilities2(locations_and_fraud_counts)
print( locations_and_fraud_prob.head() )

                         RealProb  FraudulentProb
ae                            1.0        0.200000
ae,az,abudhabi                1.0        0.333333
ae,du                         1.0        0.166667
ae,du,dubai                   1.0        0.055556
ae,du,dubaiinternetcity       1.0        0.250000


## (3) Department feature

compute fraud counts of each unique department with respect to the class lables.

In [14]:
training_data_departments = training_data['department'].str.lower()

bool_nan_dep = training_data_departments.isnull()
nan_index_dep = np.where(bool_nan_dep== True)[0]

training_data_departments[nan_index_dep] ="department unavailable"

unique_departments_new = np.unique(training_data_departments)
print("Unique departments :",unique_departments_new.shape)

Unique departments : (920,)


In [15]:
#compute fraud counts of each unique department with respect to the class labels.
departments_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_departments, unique_departments_new)
print(departments_and_fraud_counts.head())

                             Real  Fraudulent
\tcorporate shared services   0.0         1.0
(consultant)                  1.0         0.0
.net development              1.0         0.0
0                             1.0         0.0
1221                          0.0         1.0


compute probabilities of each unique department with respect to the class labels.

In [16]:
departments_and_fraud_prob = compute_probabilities2(departments_and_fraud_counts)
print( departments_and_fraud_prob.head() )

                             RealProb  FraudulentProb
\tcorporate shared services  0.333333        1.000000
(consultant)                 1.000000        0.333333
.net development             1.000000        0.333333
0                            1.000000        0.333333
1221                         0.333333        1.000000


In [17]:

# departments_and_fraud_prob = compute_probabilities(departments_and_fraud_counts)
    
# print('department [prob_department_fraud_false, prob_department_fraud_true]','\n')
# print_dictionary(departments_and_fraud_prob)


## (4) Salary Range feature

compute fraud counts of each unique value in salary_range feature with respect to the class labes.

In [18]:
training_data_salary_ranges = training_data['salary_range'].str.lower()

bool_nan_sal_ran = training_data_salary_ranges.isnull()
nan_index_sal_ran = np.where(bool_nan_sal_ran == True)[0]

training_data_salary_ranges[nan_index_sal_ran] ="salary_range unavailable"

unique_salary_ranges_new = np.unique(training_data_salary_ranges)
print("Unique salary ranges :",unique_salary_ranges_new.shape)

Unique salary ranges : (610,)


In [19]:
#compute fraud counts of each unique salary range with respect to the class labels.
salary_range_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_salary_ranges, unique_salary_ranges_new)
print(salary_range_and_fraud_counts.head())
#.loc["21-63000"]

          Real  Fraudulent
0-0       76.0         0.0
0-1        1.0         0.0
0-1000     2.0         0.0
0-100000   3.0         0.0
0-110406   1.0         0.0


compute probabilities of each unique salary range value with respect to the class labels.

In [20]:
salary_range_and_fraud_prob = compute_probabilities2(salary_range_and_fraud_counts)
print( salary_range_and_fraud_prob.head() )

          RealProb  FraudulentProb
0-0            1.0        0.012821
0-1            1.0        0.333333
0-1000         1.0        0.250000
0-100000       1.0        0.200000
0-110406       1.0        0.333333


## (5) Telecommuting feature

compute fraud counts of each unique value in telecommuting feature with respect to the class labes.

In [21]:
training_data_telecommuting = training_data['telecommuting']
telecommuting_unique = np.unique(training_data['telecommuting'])
print("Unique telecommuting:",telecommuting_unique.shape)

Unique telecommuting: (2,)


In [22]:
#compute fraud counts of each unique telecommuting boolean with respect to the class labels.
telecommuting_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_telecommuting, telecommuting_unique)
print(telecommuting_and_fraud_counts.head())

     Real  Fraudulent
0  9831.0       421.0
1   437.0        40.0


compute probabilities of each unique telecommuting value with respect to the class labels.

In [23]:
telecommuting_and_fraud_prob = compute_probabilities2(telecommuting_and_fraud_counts)
print( telecommuting_and_fraud_prob.head() )

   RealProb  FraudulentProb
0  0.958935        0.041065
1  0.916143        0.083857


## (6) Company logo feature

compute fraud counts of each unique value in has_company_logo feature with respect to the class labes.

In [24]:
training_data_company_logo = training_data['has_company_logo']
company_logo_unique = np.unique(training_data_company_logo)
print("Unique company logo:",company_logo_unique.shape)
print(company_logo_unique)

Unique company logo: (2,)
[0 1]


In [25]:
#compute fraud counts of each unique company logo boolean with respect to the class labels.
company_logo_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_company_logo, company_logo_unique)
print(company_logo_and_fraud_counts.head())

     Real  Fraudulent
0  1873.0       275.0
1  8395.0       186.0


compute probabilities of each unique has_company_logo feature value with respect to the class labels.

In [26]:
company_logo_and_fraud_prob = compute_probabilities2(company_logo_and_fraud_counts)
print( company_logo_and_fraud_prob.head() )

   RealProb  FraudulentProb
0  0.871974        0.128026
1  0.978324        0.021676


## (7) Questions feature

compute fraud counts of each unique value in has_questions feature with respect to the class labes.

In [27]:
training_data_questions = training_data['has_questions']
questions_unique = np.unique(training_data_questions)
print("Unique questions:",questions_unique.shape)
print(questions_unique)

Unique questions: (2,)
[0 1]


In [28]:
questions_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_questions, questions_unique)
print(questions_and_fraud_counts.head())

     Real  Fraudulent
0  4702.0       310.0
1  5566.0       151.0


compute probabilities of each unique has_questions feature value with respect to the class labels.

In [29]:
questions_and_fraud_prob = compute_probabilities2(questions_and_fraud_counts)
print( questions_and_fraud_prob.head() )

   RealProb  FraudulentProb
0  0.938148        0.061852
1  0.973588        0.026412


## (8) Employment Type feature.

compute fraud counts of each unique employment_type feature value with respect to the class labels.

In [30]:
training_data_employment_type = training_data['employment_type'].str.lower()

bool_nan_empl = training_data_employment_type.isnull()
nan_index_empl = np.where(bool_nan_empl == True)[0]

training_data_employment_type[nan_index_empl] ="employment type unavailable"

unique_employment_type = np.unique(training_data_employment_type)
print("Unique employment types :",unique_employment_type.shape)
print(unique_employment_type)

Unique employment types : (6,)
['contract' 'employment type unavailable' 'full-time' 'other' 'part-time'
 'temporary']


In [31]:
employment_type_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_employment_type, unique_employment_type)
print(employment_type_and_fraud_counts.head())

                               Real  Fraudulent
contract                     1040.0        26.0
employment type unavailable  2098.0       124.0
full-time                    6405.0       289.0
other                         128.0         4.0
part-time                     445.0        17.0


compute probabilities of each unique employment_type feature value with respect to the class labels.

In [32]:
employment_type_and_fraud_prob = compute_probabilities2(employment_type_and_fraud_counts)
print( employment_type_and_fraud_prob.head() )

                             RealProb  FraudulentProb
contract                     0.975610        0.024390
employment type unavailable  0.944194        0.055806
full-time                    0.956827        0.043173
other                        0.969697        0.030303
part-time                    0.963203        0.036797


## (9) Required Experience feature

compute fraud counts of each unique required_experience feature value with respect to the class labels.

In [33]:
training_data_required_experience = training_data['required_experience'].str.lower()

bool_nan_req_ex = training_data_required_experience.isnull()
nan_index_req_ex = np.where(bool_nan_req_ex == True)[0]

training_data_required_experience[nan_index_req_ex] ="required experience unavailable"

unique_required_experience = np.unique(training_data_required_experience)
print("Unique required experience :",unique_required_experience.shape)
print(unique_required_experience)

Unique required experience : (8,)
['associate' 'director' 'entry level' 'executive' 'internship'
 'mid-senior level' 'not applicable' 'required experience unavailable']


In [34]:
required_experience_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_required_experience, unique_required_experience)
print(required_experience_and_fraud_counts.head())

               Real  Fraudulent
associate    1379.0        24.0
director      232.0        11.0
entry level  1610.0        91.0
executive      80.0         8.0
internship    230.0         6.0


compute probabilities of each unique required_experience feature value with respect to the class labels.

In [35]:
required_experience_and_fraud_prob = compute_probabilities2(required_experience_and_fraud_counts)
print( required_experience_and_fraud_prob.head() )

             RealProb  FraudulentProb
associate    0.982894        0.017106
director     0.954733        0.045267
entry level  0.946502        0.053498
executive    0.909091        0.090909
internship   0.974576        0.025424


## (10) Required Education feature

compute fraud counts of each unique required_education feature value with respect to the class labels.

In [36]:
training_data_required_education = training_data['required_education'].str.lower()

bool_nan_req_ed = training_data_required_education.isnull()
nan_index_req_ed = np.where(bool_nan_req_ed == True)[0]

training_data_required_education[nan_index_req_ed] ="required education unavailable"

unique_required_education = np.unique(training_data_required_education)
print("Unique required education :",unique_required_education.shape)
print(unique_required_education)

Unique required education : (14,)
['associate degree' "bachelor's degree" 'certification' 'doctorate'
 'high school or equivalent' "master's degree" 'professional'
 'required education unavailable' 'some college coursework completed'
 'some high school coursework' 'unspecified' 'vocational'
 'vocational - degree' 'vocational - hs diploma']


In [37]:
required_education_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_required_education, unique_required_education)
print(required_education_and_fraud_counts.head())

                             Real  Fraudulent
associate degree            148.0         5.0
bachelor's degree          3192.0        62.0
certification                80.0        11.0
doctorate                    20.0         0.0
high school or equivalent  1158.0        98.0


compute probabilities of each unique required_education feature value with respect to the class labels.

In [38]:
required_education_and_fraud_prob = compute_probabilities2(required_education_and_fraud_counts)
print( required_education_and_fraud_prob.head() )

                           RealProb  FraudulentProb
associate degree           0.967320        0.032680
bachelor's degree          0.980947        0.019053
certification              0.879121        0.120879
doctorate                  1.000000        0.045455
high school or equivalent  0.921975        0.078025


## (11) Industry feature

compute fraud counts of each unique industry feature value with respect to the class labels.

In [39]:
training_data_industry = training_data['industry'].str.lower()

bool_nan_industry = training_data_industry.isnull()
nan_index_industry = np.where(bool_nan_industry == True)[0]

training_data_industry[nan_index_industry] ="industry unavailable"

unique_industry = np.unique(training_data_industry)
print("Unique industry types :",unique_industry.shape)

Unique industry types : (125,)


In [40]:
industry_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_industry, unique_industry)
print(industry_and_fraud_counts.head())

                         Real  Fraudulent
accounting               62.0        25.0
airlines/aviation        26.0         0.0
animation                 2.0         0.0
apparel & fashion        49.0         0.0
architecture & planning   6.0         0.0


compute probabilities of each unique industry feature value with respect to the class labels.

In [41]:
industry_and_fraud_prob = compute_probabilities2(industry_and_fraud_counts)
print( industry_and_fraud_prob.head() )

                         RealProb  FraudulentProb
accounting               0.712644        0.287356
airlines/aviation        1.000000        0.035714
animation                1.000000        0.250000
apparel & fashion        1.000000        0.019608
architecture & planning  1.000000        0.125000


In [42]:

# industries_and_fraud_prob = compute_probabilities(industries_and_fraud_counts)
    
# print('industry [prob_industry_fraud_false, prob_industry_fraud_true]','\n')
# print_dictionary(industries_and_fraud_prob)


## (12) Function feature

compute fraud counts of each unique function feature value with respect to the class labels.

In [43]:
training_data_function = training_data['industry'].str.lower()

bool_nan_function = training_data_function.isnull()
nan_index_function = np.where(bool_nan_function == True)[0]

training_data_function[bool_nan_function] ="industry unavailable"

unique_function = np.unique(training_data_function)
print("Unique function types :",unique_function.shape)

Unique function types : (125,)


In [44]:
functions_and_fraud_counts = compute_fraud_real_counts(training_data['fraudulent'], training_data_function, unique_function)
print(functions_and_fraud_counts.head())

                         Real  Fraudulent
accounting               62.0        25.0
airlines/aviation        26.0         0.0
animation                 2.0         0.0
apparel & fashion        49.0         0.0
architecture & planning   6.0         0.0


compute probabilities of each unique function feature value with respect to the class labels.

In [45]:
functions_and_fraud_prob = compute_probabilities2(functions_and_fraud_counts)
print(functions_and_fraud_prob.head())

                         RealProb  FraudulentProb
accounting               0.712644        0.287356
airlines/aviation        1.000000        0.035714
animation                1.000000        0.250000
apparel & fashion        1.000000        0.019608
architecture & planning  1.000000        0.125000


## Create Final Model

In [46]:

all_probabilities = []
all_probabilities.append(titles_and_fraud_prob)
all_probabilities.append(locations_and_fraud_prob)
all_probabilities.append(departments_and_fraud_prob)
all_probabilities.append(salary_range_and_fraud_prob)
all_probabilities.append(telecommuting_and_fraud_prob)
all_probabilities.append(company_logo_and_fraud_prob)
all_probabilities.append(questions_and_fraud_prob)
all_probabilities.append(employment_type_and_fraud_prob)
all_probabilities.append(required_experience_and_fraud_prob)
all_probabilities.append(required_education_and_fraud_prob)
all_probabilities.append(industry_and_fraud_prob)
all_probabilities.append(functions_and_fraud_prob)

selected_features = []
for i in range(features.shape[0]):
    if i not in range(4,8) and i != features.shape[0]:
        selected_features.append(features[i])
print(selected_features)

['title', 'location', 'department', 'salary_range', 'telecommuting', 'has_company_logo', 'has_questions', 'employment_type', 'required_experience', 'required_education', 'industry', 'function']


### Evaluate performance of model.

In [47]:

def classify_job_postings(dataset):
    
    datapoint_prob_0 = []
    datapoint_prob_1 = []
    bayes_numerator = []
    
    job_ids = dataset['job_id'].to_numpy()
    i = job_ids[0]-1
    print(i)
    
    # for each data point
    for i in range(i, i + dataset.shape[0]):
        
        datapoint = dataset.loc[i]
        #-------------------------------------------
        #print('data point:', '\n', datapoint, '\n')
        #-------------------------------------------
        
        # go through features in data
        for feature in data:
            
            value = datapoint[feature]
            #----------------------------------------
            #print('value is: ', value)
            #----------------------------------------    
            if isinstance(value, str):
                value = value.lower()

            if feature == 'location':

                if str(value) == 'nan':
                    #-------------------------------------------------
                    #if value in data[feature]:
                        #print(value, ' not in data with key', feature)
                    #-------------------------------------------------
                    if value in data[feature]:
                        #print(value, ' in data with key', feature)
                        datapoint_prob_0.append(data[feature][value][0])
                        datapoint_prob_1.append(data[feature][value][1])
                    
                else:
                    temp = value.split(',')
                    #--------------------------------------------
                    #if temp[0] not in data[feature]:
                        #print(temp[0], ' not in data with key', feature)
                    #--------------------------------------------
                    
                    # NOTE: here we just picking the first location but some data points have sets of locations so we need
                    # to consider each location of that data point.
                    
                    if temp[0] in data[feature]:
                        #print(temp[0], ' in data with key', feature)
                        datapoint_prob_0.append(data[feature][temp[0]][0])
                        datapoint_prob_1.append(data[feature][temp[0]][1])
            else:
                #----------------------------------------
                #if value not in data[feature]:
                    #print(value, ' not in data with key', feature)
                #----------------------------------------
                if value in data[feature]:
                    #print(value, ' in data with key', feature)
                    datapoint_prob_0.append(data[feature][value][0])
                    datapoint_prob_1.append(data[feature][value][1])

        #print('length of lists respectively: ', len(datapoint_prob_0), ' ', len(datapoint_prob_1))
        
        given_fraud_false = np.prod(datapoint_prob_0) * prior_prob_fraud_false
        given_fraud_true = np.prod(datapoint_prob_1) * prior_prob_fraud_true

        bayes_numerator.append(given_fraud_false)
        bayes_numerator.append(given_fraud_true)

        #print()
        print('job posting: ', datapoint['title'])
        print('bayes numerator given job posting IS NOT fraudulent: ', bayes_numerator[0])
        print('bayes numerator given job posting IS fraudulent: ', bayes_numerator[1])

        max_value = max(bayes_numerator)
        if max_value == bayes_numerator[0]:
            print('classification is: ', 0, ' not fraudulent', '\n\n')
        else:
            print('classification is: ', 1, ' fraudulent', '\n\n')
            
        # clear lists for next data point classification
        datapoint_prob_0.clear()
        datapoint_prob_1.clear()
        bayes_numerator.clear()
    

In [48]:
def clean_datapoints(data_points):
    
    data_index = data_points.index.values
    
    # Clean features
    data_points['title'] = data_points['title'].str.lower()
    data_points['location'] = data_points['location'].str.lower()
    data_points['department'] = data_points['department'].str.lower()
    data_points['employment_type'] = data_points['employment_type'].str.lower()
    data_points['required_experience'] = data_points['required_experience'].str.lower()
    data_points['required_education'] = data_points['required_education'].str.lower()
    data_points['industry'] = data_points['industry'].str.lower()
    data_points['function'] = data_points['function'].str.lower()
    
    #Remove nan
    bool_NaN = data_points['title'].isnull()
    NaN_index= np.where(bool_NaN == True)[0]
    sum_index = np.add(NaN_index,data_index[0])
    data_points['title'][sum_index] ="title unavailable"
    
    bool_NaN = data_points['location'].isnull()
    NaN_index = np.where(bool_NaN == True)[0]
    sum_index = np.add(NaN_index,data_index[0])
    data_points['location'].loc[sum_index] ="location unavailable"
    
    bool_NaN = data_points['department'].isnull()
    NaN_index = np.where(bool_NaN == True)[0]
    sum_index = np.add(NaN_index,data_index[0])
    data_points['department'].loc[sum_index] ="department unavailable"
    
    bool_NaN = data_points['salary_range'].isnull()
    NaN_index = np.where(bool_NaN == True)[0]
    sum_index = np.add(NaN_index,data_index[0])
    data_points['salary_range'].loc[sum_index] ="salary_range unavailable"
    
    bool_NaN = data_points['employment_type'].isnull()
    NaN_index = np.where(bool_NaN == True)[0]
    sum_index = np.add(NaN_index,data_index[0])
    data_points['employment_type'].loc[sum_index] ="employment type unavailable"
    
    bool_NaN = data_points['required_experience'].isnull()
    NaN_index = np.where(bool_NaN == True)[0]
    sum_index = np.add(NaN_index,data_index[0])
    data_points['required_experience'].loc[sum_index] ="required experience unavailable"
    
    bool_NaN = data_points['required_education'].isnull()
    NaN_index = np.where(bool_NaN == True)[0]
    sum_index = np.add(NaN_index,data_index[0])
    data_points['required_education'].loc[sum_index] ="required education unavailable"
    
    bool_NaN = data_points['industry'].isnull()
    NaN_index = np.where(bool_NaN == True)[0]
    sum_index = np.add(NaN_index,data_index[0])
    data_points['industry'].loc[sum_index] ="industry unavailable"
    
    bool_NaN = data_points['function'].isnull()
    NaN_index = np.where(bool_NaN == True)[0]
    sum_index = np.add(NaN_index,data_index[0])
    data_points['function'].loc[sum_index] ="function unavailable"
    
    
    return data_points

In [49]:
def feature_calc(prob_arr,feature):
    if(feature in prob_arr.index):
        prob_fraud = prob_arr.loc[feature]['FraudulentProb']
        prob_real = prob_arr.loc[feature]['RealProb']
    else:
        prob_fraud = 0
        prob_real = 0
    return prob_fraud ,prob_real

In [50]:
def retreive_probabilities(data_point):
    
    # Probabilities
    prob_fraud = np.array([])
    prob_real = np.array([])
    
    print("DATA_POINT:",data_point)
    
    # Test
    for i in range(len(all_probabilities)):
        temp_fraud, temp_real = feature_calc(all_probabilities[i],data_point[i])
        prob_fraud = np.append(prob_fraud,temp_fraud)
        prob_real = np.append(prob_real,temp_real)
    
    return prob_fraud,prob_real

In [51]:
def classify_job_postings2(dataset):
    
    # Get required features
    data_points = dataset.loc[:, selected_features]
    
    #Clean Data
    data_points = clean_datapoints(data_points)
    
    #Create an empty dataframe to hold probablities
    dfObj = pd.DataFrame(columns=['RealProb','FraudProb'], index = data_points["title"].values)
    
    # Probablities objects
    fraud_obj = pd.DataFrame(columns=selected_features, index = data_points["title"].values)
    
    for i in range(data_points.shape[0]):
        data_point = data_points.iloc[i].to_numpy()
        probability_fraud, probability_real = retreive_probabilities(data_point)
        print("Probabilities fraud:",probability_fraud)
        print("Probabilities real:",probability_real)
        print("\n")
    

In [52]:

#----------------------------------------------------------------
# just for illustration but not to be used as a way to determine the performance of the model, cause this is training data

# for training data slicing starts at 0 and ends at 10728
#classify_job_postings(training_data.loc[:20])

# NOTE: there's a job posting thats fraudulent in the range of training data specified below but its classified as 0 (non fraud), 
# however this can be due to a number of reasons but before we can dive into those we must ensure the 
# classify_data_point function is working correctly for multiple data points of each feature.

#classify_job_postings(training_data.loc[97:105])
#----------------------------------------------------------------

# for evaluating performance of model we use validation and test data

# for validation data slicing starts at 10729 and ends at 16450
# classify_job_postings2(validation_data.loc[10729:10748])
classify_job_postings2(validation_data)

# for test data slicing starts at 16451 till the end of the test data. 
#classify_job_postings(test_data.loc[16451:16470])


/home/thabo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/thabo/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


DATA_POINT: ['senior front-end engineer' 'us,tx,austin' 'web'
 'salary_range unavailable' 0 1 1 'full-time' 'mid-senior level'
 "bachelor's degree" 'computer software' 'engineering']
Probabilities fraud: [0.         0.17757009 0.25       0.03687299 0.04106516 0.0216758
 0.02641245 0.04317299 0.0301259  0.01905347 0.00367647 0.        ]
Probabilities real: [0.         0.82242991 1.         0.96312701 0.95893484 0.9783242
 0.97358755 0.95682701 0.9698741  0.98094653 0.99632353 0.        ]


DATA_POINT: ['field sales representative' 'us,fl,st.louis' 'department unavailable'
 '35000-50000' 0 1 1 'full-time' 'associate' 'unspecified'
 'hospital & health care' 'sales']
Probabilities fraud: [0.2        0.         0.0375109  0.09090909 0.04106516 0.0216758
 0.02641245 0.04317299 0.0171062  0.0284738  0.0952381  0.        ]
Probabilities real: [1.         0.         0.9624891  0.90909091 0.95893484 0.9783242
 0.97358755 0.95682701 0.9828938  0.9715262  0.9047619  0.        ]


DATA_POINT: ['fie

Probabilities fraud: [0.         0.2        0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.04317299 0.04526749 0.01905347 0.0157563  0.        ]
Probabilities real: [0.         1.         0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.95682701 0.95473251 0.98094653 0.9842437  0.        ]


DATA_POINT: ['customer service associate' 'us,fl,tampa' 'department unavailable'
 'salary_range unavailable' 0 1 0 'full-time' 'entry level'
 'high school or equivalent' 'insurance' 'customer service']
Probabilities fraud: [0.00729927 0.13636364 0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.04317299 0.05349794 0.07802548 0.08064516 0.        ]
Probabilities real: [1.         0.86363636 0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.95682701 0.94650206 0.92197452 0.91935484 0.        ]


DATA_POINT: ['beauty & fragrance consultants needed' 'gb,london'
 'department unavailable' 'salary_range unavailable' 0 1 0
 'employment type unavailable' 'required experience 

Probabilities fraud: [0.01515152 0.01216545 0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.03679654 0.05349794 0.07802548 0.03752345 0.        ]
Probabilities real: [1.         0.98783455 0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.96320346 0.94650206 0.92197452 0.96247655 0.        ]


DATA_POINT: ['service delivery manager 1' 'us,il,chicago' 'department unavailable'
 'salary_range unavailable' 0 1 0 'full-time' 'mid-senior level'
 "bachelor's degree" 'financial services' 'management']
Probabilities fraud: [0.07142857 0.01360544 0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.04317299 0.0301259  0.01905347 0.03752345 0.        ]
Probabilities real: [1.         0.98639456 0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.95682701 0.9698741  0.98094653 0.96247655 0.        ]


DATA_POINT: ['customer service associate - part time' 'us,fl,tampa'
 'department unavailable' 'salary_range unavailable' 0 1 0 'part-time'
 'entry level' 'high school or equi

Probabilities fraud: [0.00729927 0.01216545 0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.04317299 0.05349794 0.07802548 0.02631579 0.        ]
Probabilities real: [1.         0.98783455 0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.95682701 0.94650206 0.92197452 0.97368421 0.        ]


DATA_POINT: ['customer service associate' 'us,ct,hartford' 'department unavailable'
 'salary_range unavailable' 0 1 0 'full-time' 'entry level'
 'high school or equivalent' 'facilities services' 'customer service']
Probabilities fraud: [0.00729927 0.11111111 0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.04317299 0.05349794 0.07802548 0.01694915 0.        ]
Probabilities real: [1.         0.88888889 0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.95682701 0.94650206 0.92197452 1.         0.        ]


DATA_POINT: ['head of supply' 'gb,lnd' 'landlord' 'salary_range unavailable' 0 1 0
 'employment type unavailable' 'required experience unavailable'
 'required educ

...

DATA_POINT: ['entry level business analyst' 'us,ga,atlanta' 'department unavailable'
 'salary_range unavailable' 0 1 0 'employment type unavailable'
 'required experience unavailable' 'required education unavailable'
 'industry unavailable' 'function unavailable']
Probabilities fraud: [0.         0.04347826 0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.05580558 0.05521327 0.04747583 0.03917595 0.        ]
Probabilities real: [0.         0.95652174 0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.94419442 0.94478673 0.95252417 0.96082405 0.        ]


DATA_POINT: ['senior marketing strategist and analyst' 'us,ny,newyorl'
 'department unavailable' '65000-125000' 0 0 1 'full-time' 'associate'
 "bachelor's degree" 'retail' 'marketing']
Probabilities fraud: [0.         0.         0.0375109  0.         0.04106516 0.12802607
 0.02641245 0.04317299 0.0171062  0.01905347 0.03937008 0.        ]
Probabilities real: [0.         0.         0.9624891  0.         0.95893484 0.87197

Probabilities fraud: [0.         0.33333333 0.0375109  0.03687299 0.04106516 0.0216758
 0.02641245 0.04317299 0.0301259  0.01905347 0.04166667 0.        ]
Probabilities real: [0.         1.         0.9624891  0.96312701 0.95893484 0.9783242
 0.97358755 0.95682701 0.9698741  0.98094653 1.         0.        ]


DATA_POINT: ['digital interns' 'ie,dublin' 'digital experience'
 'salary_range unavailable' 0 1 0 'temporary'
 'required experience unavailable' 'required education unavailable'
 'airlines/aviation' 'marketing']
Probabilities fraud: [0.         0.03225806 0.         0.03687299 0.04106516 0.0216758
 0.06185156 0.00653595 0.05521327 0.04747583 0.03571429 0.        ]
Probabilities real: [0.         1.         0.         0.96312701 0.95893484 0.9783242
 0.93814844 0.99346405 0.94478673 0.95252417 1.         0.        ]


DATA_POINT: ['residential property manager' 'us,ma,boston' 'property management'
 'salary_range unavailable' 0 1 0 'full-time' 'associate'
 "bachelor's degree" 'real 

Probabilities fraud: [0.         0.03846154 0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.04317299 0.0171062  0.01905347 0.0952381  0.        ]
Probabilities real: [0.         1.         0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.95682701 0.9828938  0.98094653 0.9047619  0.        ]


DATA_POINT: ['marketing manager' 'us,ny,newyork' 'department unavailable'
 'salary_range unavailable' 0 1 0 'full-time' 'associate'
 'required education unavailable' 'computer software' 'marketing']
Probabilities fraud: [0.03030303 0.01216545 0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.04317299 0.0171062  0.04747583 0.00367647 0.        ]
Probabilities real: [1.         0.98783455 0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.95682701 0.9828938  0.95252417 0.99632353 0.        ]


DATA_POINT: ['head of product' 'gb,lnd' 'department unavailable'
 'salary_range unavailable' 0 1 0 'full-time'
 'required experience unavailable' 'required education unavailable'


Probabilities fraud: [0.         0.03448276 0.25       0.03687299 0.04106516 0.0216758
 0.02641245 0.05580558 0.05521327 0.04747583 0.03917595 0.        ]
Probabilities real: [0.         1.         1.         0.96312701 0.95893484 0.9783242
 0.97358755 0.94419442 0.94478673 0.95252417 0.96082405 0.        ]


DATA_POINT: ['process engineer' 'us,il,chicago' 'department unavailable'
 'salary_range unavailable' 0 0 0 'full-time'
 'required experience unavailable' 'required education unavailable'
 'industry unavailable' 'function unavailable']
Probabilities fraud: [0.125      0.01360544 0.0375109  0.03687299 0.04106516 0.12802607
 0.06185156 0.04317299 0.05521327 0.04747583 0.03917595 0.        ]
Probabilities real: [1.         0.98639456 0.9624891  0.96312701 0.95893484 0.87197393
 0.93814844 0.95682701 0.94478673 0.95252417 0.96082405 0.        ]


DATA_POINT: ['overnight caregiver for a gregarious man' 'us,or,seportland'
 'department unavailable' 'salary_range unavailable' 0 1 1 'full-t

Probabilities real: [0.         0.97272727 0.97457627 0.96312701 0.95893484 0.9783242
 0.97358755 0.95682701 0.9698741  0.98094653 0.99632353 0.        ]


DATA_POINT: ['full-stack engineer' 'us' 'department unavailable'
 'salary_range unavailable' 0 1 0 'full-time' 'mid-senior level'
 'required education unavailable' 'industry unavailable' 'engineering']
Probabilities fraud: [0.         0.0483871  0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.04317299 0.0301259  0.04747583 0.03917595 0.        ]
Probabilities real: [0.         0.9516129  0.9624891  0.96312701 0.95893484 0.9783242
 0.93814844 0.95682701 0.9698741  0.95252417 0.96082405 0.        ]


DATA_POINT: ['staff accountant' 'us,ny,newyork' 'department unavailable'
 'salary_range unavailable' 0 1 0 'full-time' 'associate'
 "bachelor's degree" 'marketing and advertising' 'accounting/auditing']
Probabilities fraud: [0.2        0.01216545 0.0375109  0.03687299 0.04106516 0.0216758
 0.06185156 0.04317299 0.0171062  0.01905